In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')

In [ ]:
train.info()

In [ ]:
##Drop row_id
train = train.drop('row_id',axis=1)
train.info()

In [ ]:
##For direction
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['direction'])
train['direction_le'] = le.transform(train['direction'])
train.info()

In [ ]:
#Drop direction
train = train.drop('direction',axis=1)
train.info()

In [ ]:
train.time.value_counts()

In [ ]:
train['time_le'] = pd.to_datetime(train['time'],format='%Y/%m/%d %H:%M:%S')
train['hour'] = train['time_le'].dt.hour
train['minute'] = train['time_le'].dt.minute
train['weekday'] = train['time_le'].dt.weekday
train.info()

In [ ]:
X = train.drop(['congestion','time_le','time'],axis=1).reset_index(drop=True)
y = train.congestion.reset_index(drop=True)
X = X.sort_values(['x','y','weekday','hour','minute','direction_le'])
X.info()

In [ ]:
'''def model():
    from xgboost import XGBRegressor
    from sklearn.ensemble import RandomForestRegressor
    model = XGBRegressor (max_depth = 10,
                                            n_estimators = 1000,
                                            min_child_weight = 5, 
                                            subsample = 0.6, 
                                           alpha = 0.1,
                                           eta = 0.1, 
                                           seed = 42)
    model1 = RandomForestRegressor(max_depth=2, random_state=0)
    
    return model'''

In [ ]:
'''from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

tscv = TimeSeriesSplit(n_splits=2, test_size=65*5)
for train_index, test_index in tscv.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    regr = model()
    regr.fit(X_train, y_train)
    prediction = regr.predict(X_test)
    print(mean_absolute_error(y_test, prediction))'''

In [ ]:
test['time_le'] = pd.to_datetime(test['time'],format='%Y/%m/%d %H:%M:%S')
test['hour'] = test['time_le'].dt.hour
test['minute'] = test['time_le'].dt.minute
test['weekday'] = test['time_le'].dt.weekday
test.info()

In [ ]:
test_X = test.drop(['time_le','time'],axis=1).reset_index(drop=True)
test_X.info()

In [ ]:
test_X = test_X.drop('row_id',axis=1)
test_X['direction_le'] = le.transform(test_X['direction'])
test_X = test_X.drop('direction',axis=1)
test_X.info()

In [ ]:
mean_train = train.groupby(['x','y','weekday','hour','minute','direction_le']).median()

In [ ]:
mean_train_cols = mean_train.reset_index()
prediction = pd.merge(test_X,mean_train_cols,'inner',on=['weekday','hour','minute','direction_le','x','y'])['congestion'].round()

In [ ]:
#prediction = regr.predict(test_X[X.columns])
prediction

In [ ]:
submission = pd.DataFrame({'row_id':test.row_id,'congestion':prediction})

In [ ]:
submission.to_csv('submission.csv', index=False)